In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pangeo_forge_cordex import logon, recipe_inputs_from_iids

In [22]:
host = "esgf-data.dkrz.de"
host = "esgf-data1.ceda.ac.uk"
# ssl = logon(host=host)

In [23]:
def logon(host=None):
    from pyesgf.logon import LogonManager

    if host is None:
        host = "esgf-data.dkrz.de"
    lm = LogonManager(verify=True)

    lm.logon(
        hostname=host,
        interactive=True,
        bootstrap=True,
    )

    print(f"logged on: {lm.is_logged_on()}")

    # create SSL context
    sslcontext = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    sslcontext.load_verify_locations(capath=lm.esgf_certs_dir)
    sslcontext.load_cert_chain(lm.esgf_credentials)
    return sslcontext

In [24]:
logon(host)

Enter myproxy username: 

 g300046
Enter password for g300046:  ········


TimeoutError: [Errno 60] Operation timed out